# Notebook for implementing a DP solution for the `0-1 knapsack problem`

## Intro

- The `knapsack problem` is based on the idea of fitting objects into a knapsack (i.e. bag)
- The idea is to get the most value in the bag, but keep it under the weight the bag can carry
    - This means there are 3 variables to consider
        1. Bag weight capacity ($W$)
        2. Item weight ($w$)
        3. Item value ($v$)
    - There are also some constraints to consider:
        1. Items cannot be broken
        2. Sum of selected item weights must be =< $B_w$
- This problem can ends up having repetitive sub-problems (as we will see later on) which makes it a possible candidate for DP (spoiler: it definitely is)


# Code solutions

In [1]:
import time
import tracemalloc

def LogPerf(func):
    """
    Decorator function to log the time it takes to complete a function
    """
    
    def wrapper(*args,**kwargs):
        # Store the starting time
        ts = time.time()

        # Run the function
        res = func(*args, **kwargs)
        
        # Check the memory usage
        current, peak = tracemalloc.get_traced_memory()

        # Print out the time
        print(" ")
        print(f"Function: {func.__name__}")
        # print("Positional args:", *args)
        # print("Keyword args:   ", **kwargs)
        print("-"*38)
        
        print(f"Time elapsed:\t\t{time.time() - ts: 0.4} s")
        print(f"Memory usage:\t\t {current/10*6:0.4} Mbs")
        print(f"Peak Memory usage:\t {peak/10*6:0.4} Mbs")
        print("+"*38)
        
        return res
    
    return wrapper

## Recursive solution  

In [2]:
@LogPerf
def TabKnap(weights, values, W):
    """
    Iterative DP implementation of 0-1 Knapsack algorithm using tabulation 
    
    NOTE: Much of this code is from external sources with some edits by me to help me better understand the flow
        Credit to GeeksforGeeks, askpython, and Holczer Balazs
    """
    
    max_len = len(weights)
    
    # Sanity check
    if max_len != len(values) or max_len < 1:
        return None
    
    # Make the table; rows = n_weights / cols = sum_weights
    table = [[0 for x in range(W+1)] for i in range(max_len)]
    
    """
    I think that we are going to want iterate between all the possible weights and see
        the max value; we don't iterate through the items; but iterate through the 
        weights instead
    """
    
    for i in range(max_len):
        # Iterate through each item
        
        for w in range(W+1):
            # Then through each weight possibility
            
            if weights[i] <= w: # If weight of item i, can fit in at this point
                # Check the two cases; shoule we add it in, or not
                
                #case1 = do not add item
                case1 = table[i-1][w] # this accesses the table at a specific weight and
                                        # index; the index is the number of items 
                                        # included
                        
                # case2 = add item
                case2 = values[i] + table[i-1][w-weights[i]]
                
                """
                Here, if we want to add the value of our current item to the position
                    in the table where we have two things:
                        1) index -1
                        2) Weight - current weight
                        
                The position indicated by these two parameters is the previous maximum
                    not including this weight (since table[i][w] = this position.
                
                NOTE: We are iterating through possible weights here, not adding up 
                    weights, that's been the part I keep forgetting to keep in mind
                """
                
                # Add in the case with the higher value
                table[i][w] = max(case1,case2)
                
            else: # The weight of the item is higher than the capacity
                table[i][w] = table[i-1][w]
                
                """
                Here we say that for the new index, the value is the same as 
                    index-1 and the weight stays the same since we have not added in a
                    new item
                """
                
    return table[max_len-1][W]

In [3]:
@LogPerf
def RecKnap(weights, values, W, out=False):
    """
    Recursive implementation of the Knapsack problem
    """
    
    # Sanity check
    n = len(weights)
    
    if n == 0:
        return None
    if W == 0:
        return None
    if n != len(values):
        return None
    
    # Make the main function
    def main(W, n):
        """
        Main function to recruse through
        """
        
        # Define base-case(s)
        if n == 0:
            return 0
        if W == 0:
            return 0
        
        # Check if the weight of the current item is greater than remaining weight (W)
        if weights[n-1] > W: # return the value from previous iteration
            return main(W, n-1)
        else: # Check if we should add the value or not
            
            if out:
                case1 = values[n-1] + main(W-weights[n-1], n-1)
                case2 = main(W, n-1)
                print(f"{n}) C1: {case1} -- C2: {case2}")
                
                return max(case1,case2)
            
            return max(values[n-1] + main(W-weights[n-1], n-1), # case1 (add item)
                       main(W, n-1)) # case 2 (do not add)
            
            
    
    return main(W,n)

In [4]:
weights = [10,12,14,6,30]
values = [20, 22, 30, 4, 40]
W = 30

RecKnap(weights,values,W, out=True)

1) C1: 20 -- C2: 0
1) C1: 20 -- C2: 0
1) C1: 20 -- C2: 0
2) C1: 42 -- C2: 20
3) C1: 50 -- C2: 42
1) C1: 20 -- C2: 0
2) C1: 22 -- C2: 20
1) C1: 20 -- C2: 0
1) C1: 20 -- C2: 0
2) C1: 42 -- C2: 20
3) C1: 52 -- C2: 42
4) C1: 54 -- C2: 52
5) C1: 40 -- C2: 54
 
Function: RecKnap
--------------------------------------
Time elapsed:		 0.0002179 s
Memory usage:		 0.0 Mbs
Peak Memory usage:	 0.0 Mbs
++++++++++++++++++++++++++++++++++++++


54

## Dynamic programming approach

- Now that we see how the recursive approach works AND how the subproblem structure looks, let's implement a DP approach built on the recursive approach
    - For any given n, the optimal solution is the same i.e. at n=1 the optimal solution is always 20 above
    - This is because n=1 means we are only considering the the first value, n=2 means we are considering the first 2 values, etc...
- Given the repetitive sub-problem structure, DP should work great
    - We will use both tabulation (faster but more complex) and memoization (slower but much simpler to implement) with our DP approaches

In [5]:
@LogPerf
def MemKnap(weights, values, W):
    """
    Recursive implementation of the Knapsack algorithm using memoization
    """
    
    # Sanity check
    n = len(weights)
    
    if n == 0:
        return None
    if W == 0:
        return None
    if n != len(values):
        return None
    
    # Make the memo
    memo = {} # {(W,i):value}
    
    # Make the main function
    def main(W, n):
        """
        Main function to recruse through
        """
        
        # Define base-case(s)
        if n == 0:
            return 0
        if W == 0:
            return 0
        
        
        # Check the memo for the values
        if (W,n-1) in memo:
                prev = memo[(W,n-1)]
        else:
            prev = main(W, n-1)
            
        # Check if the weight of the current item is greater than remaining weight (W)
        if weights[n-1] > W: # return the value from previous iteration
            return prev
        else: # Check if we should add the value or not
            
            
                
            memo[(W, n)] = max(values[n-1] + main(W-weights[n-1], n-1), # case1 (add item)
                       prev) # case 2 (do not add)
            return memo[(W, n)]
            
                
    return main(W,n)

In [6]:
RecKnap(weights,values,W)

 
Function: RecKnap
--------------------------------------
Time elapsed:		 0.0003641 s
Memory usage:		 0.0 Mbs
Peak Memory usage:	 0.0 Mbs
++++++++++++++++++++++++++++++++++++++


54

In [8]:
MemKnap(weights,values,W)

 
Function: MemKnap
--------------------------------------
Time elapsed:		 0.0002367 s
Memory usage:		 0.0 Mbs
Peak Memory usage:	 0.0 Mbs
++++++++++++++++++++++++++++++++++++++


54

In [9]:
from random import randint

In [14]:
# Randomly generate some data and test it
    # We will use the TabKnap function as our gold-standard since we know it works (it's from an external source)
weights = [randint(0,100) for x in range(30)]
values = [randint(0,100) for x in range(30)]

W = 250

In [15]:
print(RecKnap(weights,values,W))
print(TabKnap(weights, values, W))
print(MemKnap(weights,values,W))

 
Function: RecKnap
--------------------------------------
Time elapsed:		 1.317 s
Memory usage:		 0.0 Mbs
Peak Memory usage:	 0.0 Mbs
++++++++++++++++++++++++++++++++++++++
798
 
Function: TabKnap
--------------------------------------
Time elapsed:		 0.002093 s
Memory usage:		 0.0 Mbs
Peak Memory usage:	 0.0 Mbs
++++++++++++++++++++++++++++++++++++++
798
 
Function: MemKnap
--------------------------------------
Time elapsed:		 0.006211 s
Memory usage:		 0.0 Mbs
Peak Memory usage:	 0.0 Mbs
++++++++++++++++++++++++++++++++++++++
798


# Summary

- The recursive implementation was never going to be the fastest here, but I always like to start from there with complicated algorithms
    - I find that it is much easier to follow the logic-flow using a recursive implementation
- As expected, the tabulation approach was faster, but it definitely felt more complicated to me as I was implementing it

In [136]:
%%timeit
patts = ["cat", "dog", "xxx", "test"]
s = "xxxxxcatdangtest"

for patt in patts:
    if patt not in s:
        patts.remove(patt)

478 ns ± 28.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [137]:
%%timeit
patts = ["cat", "dog", "xxx", "test"]
s = "xxxxxcatdangtest"

[patts.remove(patt) for patt in patts \
 if patt not in s]

750 ns ± 27.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
